In [1]:
from PIL import Image
import torch
from datasets import load_dataset, DatasetInfo
import numpy as np
from collections import Counter, defaultdict
import random

from resnext import ResNeXt
from resnext.data import ImageNetDataset, get_val_transforms, get_train_transforms, get_collate_function, Collate
from resnext.utils import create_image_grid

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
batch_size = 128
mixup_alpha=0.2
cutmix_alpha=1.0
num_workers = 0

In [5]:

val_dataset = ImageNetDataset(split="validation", transform = get_val_transforms())
train_dataset = ImageNetDataset(split="train", transform = get_train_transforms())

Loading dataset shards:   0%|          | 0/257 [00:00<?, ?it/s]

In [6]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle = True,
    pin_memory=True,
    drop_last=True,
    num_workers=num_workers,
    collate_fn=get_collate_function(num_classes=train_dataset.num_classes, mixup_alpha=mixup_alpha, cutmix_alpha=cutmix_alpha),
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=num_workers,
    collate_fn=Collate(),
)

In [7]:
train_batch = next(iter(train_dataloader))

In [8]:
val_batch = next(iter(val_dataloader))

In [9]:
images = val_batch["image"]
pred_labels = val_batch["class_name"][::-1]
gt_labels = val_batch["class_name"]
image_array = create_image_grid(images, pred_labels, gt_labels, max_images=50)

In [16]:
image = Image.fromarray(image_array)